<a href="https://colab.research.google.com/github/QiqiLian/Analyzer/blob/master/CSE390_Spring2024_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preface: Google Colab

For this assignment, we will use Google Colab. This will allow us to use resources like GPUs that may not be available on our personal computers, and ensure identical use of external dependencies for everybody. You can use your Stony Brook (`*.stonybrook.edu`) account and save the results on Google Drive.

To get started, go to the [Google Colab Notebook](https://colab.research.google.com/notebooks/). There, you'll find a tutorial called *Overview of Colaboratory Features*, which provides a quick overview on using Google Colab. There is also this concise [Google Colab Tutorial](https://colab.research.google.com/drive/16pBJQePbqkz3QFV54L4NIkOn1kwpuRrj).

Colab is built on top of Jupyter notebook. But remember, inactivity will disconnect your runtime environment.

## Libraries

You are encouraged to use the following libraries as and when needed. This assignment is as much about learning to use available tools, as it is about understanding how to implement simple aspects of computational finance.

- **NumPy**: the most important library for numerical computations in Python; provides support for arrays and matrices; has a large collection of high-level mathematical functions used in linear algebra and probability/statistics.
- **Pandas**: for high-performance, easy-to-use data structures and data analysis; particularly good for data manipulation and analysis; offers `DataFrame` objects, which are ideal for time-series data.
- **SciPy** or **PyTorch**: for optimization, regression, interpolation, etc.
- **Statsmodels**: statistical modeling library built on top of NumPy and SciPy.
- **Matplotlib** and **Seaborn**: for plotting and visualization.

# American Put Options (20 points)

Write your solution to the first question (**American Put Options**) in the cell below. You are free to have additional methods, but the user should be able to call only the `price_american_put_options` function to obtain the results corresponding to each line in the input text or file.

In [12]:
import numpy as np

def price_american_put_options(input_text: str | bytes) -> None:
    # Check if input_text is the path to a file
    if isinstance(input_text, str) and '\n' not in input_text:
        with open(input_text, 'r') as file:
            lines = file.read().strip().split('\n')
    else:
        lines = input_text.strip().split('\n')

    for line in lines:
        params = line.split('\t')
        risk_free_rate = float(params[0])
        time = float(params[1])
        num_steps = int(params[2])
        volatility = float(params[3])
        initial_price = float(params[4])
        strike_price = float(params[5])
        print(value_of_american_put_option(risk_free_rate, time, num_steps, volatility, initial_price, strike_price))

def value_of_american_put_option(risk_free_rate: float, time: float, num_steps: int, volatility: float, initial_price: float, strike_price: float) -> float:
    # Calculate parameters for the binomial tree
    dt = time / num_steps
    u = np.exp(volatility * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(risk_free_rate * dt) - d) / (u - d)

    # Initialize arrays to hold the stock prices and option values
    stock_prices = np.zeros(num_steps + 1)
    option_values = np.zeros(num_steps + 1)

    # Populate the stock prices at maturity
    for i in range(num_steps + 1):
        stock_prices[i] = initial_price * (u ** (num_steps - i)) * (d ** i)
        option_values[i] = max(0, strike_price - stock_prices[i])

    # Recursive calculation of option values
    for i in range(num_steps - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = max(strike_price - stock_prices[j],
                                   np.exp(-risk_free_rate * dt) * (p * option_values[j] + (1 - p) * option_values[j + 1]))
        # No need to consider early exercise for a down step as the stock price is lower

    return option_values[0]


# Asian Call Options (20 points)

Write your solution to the second question (**Asian Call Options**) in the cell below. You are free create additional functions, but the user must be able to call `price_asian_call_options` to obtain the results corresponding to each line in the input text or file.

In [19]:
import numpy as np

def price_asian_call_options(input_text: str | bytes) -> None:
    lines = input_text.strip().split('\n') if isinstance(input_text, str) else input_text.decode().strip().split('\n')
    for line in lines:
      params = line.split('\t')
      risk_free_rate = float(params[0])
      time = float(params[1])
      num_steps = int(params[2])  # Convert to int here
      volatility = float(params[3])
      initial_price = float(params[4])
      strike_price = float(params[5])
      print(value_of_american_put_option(risk_free_rate, time, num_steps, volatility, initial_price, strike_price))


def calculate_asian_call_option_price(risk_free_rate, time, num_steps, volatility, initial_price, strike_price):
    dt = time / num_steps
    u = np.exp(volatility * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(risk_free_rate * dt) - d) / (u - d)

    # Initialize the binomial tree
    stock_price_tree = np.zeros((num_steps + 1, num_steps + 1))
    for i in range(num_steps + 1):
        for j in range(i + 1):
            stock_price_tree[j, i] = initial_price * (u ** (i - j)) * (d ** j)

    # Initialize the option value tree
    option_value_tree = np.zeros((num_steps + 1, num_steps + 1))

    # Calculate the option value at each node
    for i in range(num_steps + 1):
        running_average = (initial_price * (i + 1) + np.cumsum(stock_price_tree[:i + 1, i])) / (i + 2)
        option_value_tree[:i + 1, i] = np.maximum(running_average - strike_price, 0)

    # Backward induction for option price
    for i in range(num_steps - 1, -1, -1):
        for j in range(i + 1):
            expected_value = np.exp(-risk_free_rate * dt) * (p * option_value_tree[j, i + 1] + (1 - p) * option_value_tree[j + 1, i + 1])
            running_average = (initial_price * (i + 1) + np.cumsum(stock_price_tree[:j + 1, i])) / (i + 2)
            option_value_tree[j, i] = max(np.maximum(running_average - strike_price, 0), expected_value)

    return option_value_tree[0, 0]
    pass  # TODO

# Linear Time Series Models

Start by completing the `README` section below, which should provide a precise explanation of how to run your code on the given data (including providing the input tab-separated file).

**Note 1:** At no point should the user/grader need to change anything in your code. For example, the input file should not be hard-coded in your program!

**Note 2:** Your `README` should very clearly mention exactly which function the user should call, and how the user should provide the tab-separated file as input.

**Note 3:** Your implementation must be supported by proper documentation in the form of docstrings, as shown in earlier cells where the function signatures were already provided. (**5 points**)

### README (5 points)

*TODO*

### AR($p$) and MA($q$) implementation (20 points)

In [ ]:
# TODO: Your AR and MA model implementations go here

### ACF and PACF visualization (10 points)

In [ ]:
# TODO: write the code to generate two plots (one for ACF and one for PACF).

### The Simple Autoregressive Model AR($p$) (10 points)

- TODO: clearly state your choice of $p$
- TODO: clearly state your estimated parameter values (for the sake of easier interpretaion, please use the same symbols we have used in our lectures)
- TODO: How did you arrive at this choice of $p$? Provide a precise description of your process and a brief justification.


### The Moving Average Model MA($q$) (10 points)

- TODO: clearly state your choice of $q$
- TODO: clearly state your estimated parameter values (for the sake of easier interpretaion, please use the same symbols we have used in our lectures)
- TODO: How did you arrive at this choice of $q$? Provide a precise description of your process and a brief justification.

# Collaboration and Resource Use Policy

You may discuss any details of this assignment at a conceptual level with anyone. In fact, discussion of ideas and helping each other to gain a better understanding of the concepts and the mathematical principles is encouraged. But any written answers (natural language or programming language) must be entirely your own original work.

You are also encouraged to use the libraries mentioned at the start of this notebook. The use of online resources (e.g., StackOverflow) that can help you understand specific functions in these libraries is also encouraged.

There must not be any collaboration in programming (including the design, implementation, and debugging of code). There must not be any code in your submission that is written by anyone other than you (whether human or AI). Submitted code will be checked against other submissions AND against AI-generated code, and evidence of plagiarism will lead to academic dishonesty charges.

# What to submit?

**This assignment is due by 11:59 pm, APR 30 (Tuesday)**

1. Make your colab notebook publicly accessible. You can do this by clicking "Share" on the top-right corner of your notebook and make sure "anyone with the link" can view your notebook. Also make sure that viewers are allowed to download your notebook. Then, **put this link in the comment section of your submission on Brightspace**.

2. Create an empty folder (locally, on your computer) called `firstname-lastname-cse390-hw3`. For example, John Doe will create `john-doe-cse354-hw3`.
    * Download this colab notebook with all the questions (code as well as the text questions) implemented/answered. This will be a single Python notebook, as a `.ipynb` file. Put the notebook in your folder.
    
    Zip this folder (i.e., create `firstname-lastname-cse390-hw3.zip`) and submit on Brightspace.

Once unzipped, your submission is expected to have the following structure:

```
john-doe-cse354-hw1
└── CSE390-Spring2024-HW3.ipynb
```